In [1]:
# This won't work until we have a package
import traitlets as T

import ipywidgets as W
from ipyradiant import LoadWidget, QueryWidget, RDFVisualization
from rdflib import BNode, Graph

In [2]:
class RadiantObject(W.Tab):
    graph = T.Instance(Graph)  # allow_none=True
    graph_id = T.Instance(BNode)
    load_widget = T.Instance(LoadWidget)
    query_widget = T.Instance(QueryWidget)
    vis_widget = T.Instance(RDFVisualization)
    selected_view_mode = T.Unicode()
    output = W.Output()

    def __init__(self, graph: Graph = None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        T.link((self.load_widget, "graph"), (self, "graph"))
        T.link((self, "graph"), (self.query_widget, "graph"))
        T.link((self, "selected_view_mode"),(self.vis_widget, "selected_view_mode"))

        if graph:
            self.graph = graph
            self.graph_id = graph.identifier

        self.children = [self.load_widget, self.query_widget, self.vis_widget]
        self.set_title(0, "RDF Loader")
        self.set_title(1, "Query Panel")
        self.set_title(2, "Vis Panel")

    
    @T.default("selected_view_mode")
    def make_selected_view_mode(self):
        return "full graph"
    
    @T.default("graph")
    def make_default_graph(self):
        return Graph()

    @T.default("graph_id")
    def make_default_graph_id(self):
        return self.graph.identifier

    @T.default("load_widget")
    def make_default_load_widget(self):
        load_widget = LoadWidget()
        return load_widget

    @T.default("query_widget")
    def make_default_query_widget(self):
        query_widget = QueryWidget(self.graph)
        return query_widget
    
    @T.default("vis_widget")
    def make_vis_widget(self):
        vis_widget = RDFVisualization(rdf_graph=self.graph)
        return vis_widget
    
     
    @T.observe("selected_view_mode","graph")
    @output.capture()   
    def update_vis_panel(self,*_):
        print('change occured')
        print(self.selected_view_mode)
        self.vis_widget = self.make_vis_widget()
        self.children =[self.load_widget, self.query_widget, self.vis_widget]






In [5]:
# Initial example of combining the widgets
ro = RadiantObject()

In [8]:
ro

RadiantObject(children=(LoadWidget(children=(LoadBox(children=(Label(value='Click to load graph file (.ttl):')…

In [9]:
ro.vis_widget.cyto_widget

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [4]:
RadiantObject.output

Output(outputs=({'name': 'stdout', 'text': 'change occured\nfull graph\n', 'output_type': 'stream'},))